# Data Preparation

## Packages

In [1]:
import os
import winsound
import numpy as np
import pandas as pd
import datetime as dt
from tqdm import tqdm

import math, collections
from scipy.stats import linregress

from matplotlib import pyplot as plt

import warnings
warnings.filterwarnings('ignore')

In [2]:
def get_slope_s(y_axis):
    global window_s_ma
    x_axis = []
    for i in range(window_s_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)

def get_slope_l(y_axis):
    global window_l_ma
    x_axis = []
    for i in range(window_l_ma):
        x_axis.append(1 + ((i+1) * 0.0001 * 0.1))
    
    slope_tick, intercept, _, _, _ = linregress(x_axis, y_axis)
    slope_tick = math.degrees(math.atan(slope_tick))
    
    return(slope_tick)


def get_month(row):
    month_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().month
    return(month_val)

def get_day(row):
    day_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').date().day
    return(day_val)   

def get_hour(row):
    hour_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().hour
    return(hour_val)   

def get_min(row):
    min_val = dt.datetime.strptime(row['TS'], '%Y-%m-%d %H:%M').time().minute
    return(min_val)   


def get_dow(row):
    dow = dt.datetime.strptime(df['TS'][0], '%Y-%m-%d %H:%M').weekday()
    return(dow)

In [3]:
def chunk_ticks(df, number_of_ticks):   
    global pip_diff
    
    df['tick'] = (df['Bid'] + df['Ask'])/2
    df = df[['tick']]

    temp_df = pd.DataFrame()
    open_list = []
    high_list = []
    low_list = []
    close_list = []


    for i in tqdm(range(0,len(df),number_of_ticks)):
        chunk_list = list(df['tick'][i:i+number_of_ticks])

        open_val = chunk_list[0]
        high_val = np.max(chunk_list)
        low_val = np.min(chunk_list)
        close_val = chunk_list[-1]

        open_list.append(chunk_list[0]) 
        high_list.append(np.max(chunk_list))
        low_list.append(np.min(chunk_list))
        close_list.append(chunk_list[-1]) 

    temp_df['open'] = open_list
    temp_df['high'] = high_list
    temp_df['low'] = low_list    
    temp_df['close'] = close_list    
    
    return(temp_df)

## Parameters

In [18]:
#pd.DataFrame(np.random.rand(14,4), columns=['a', 'b', 'c', 'd'])

year = 2019
source_file_path = f'data/tick_{year}.csv'

number_of_ticks = 60
pip_diff = 0.0001

diff_col = 'Close'

rsi_window = 10
rs_max = 1e6

window_s_ma = 10
window_l_ma = 50

## Read data

In [34]:
%%time
path, file_name = os.path.split(source_file_path)

target_file_name = 'tab_'+file_name
target_file_path = os.path.join(path, target_file_name)

df1 = pd.read_csv(source_file_path)

Wall time: 17 s


In [35]:
df = df1[0:50000]

In [36]:
print(f'Total recs : {len(df)}')
df = chunk_ticks(df, number_of_ticks)
print(f'Reduced recs : {len(df)}')
df.head()

100%|███████████████████████████████████████████████| 834/834 [00:00<00:00, 7426.81it/s]

Total recs : 50000
Reduced recs : 834


,open,high,low,close
0,1.14640,1.146700,1.146220,1.146700
1,1.14670,1.146740,1.146630,1.146685
2,1.14669,1.146770,1.146400,1.146435
3,1.14644,1.146440,1.146215,1.146220
4,1.14623,1.146415,1.146220,1.146260


## Data manipulation

In [37]:
df['target_gain'] = abs(df['high'].shift(-1) - df['close'])
df['target_loss'] = abs(df['low'].shift(-1) - df['close'])

df['direction'] = 'same'
df.loc[(df['target_gain'] > df['target_loss']) & (df['target_gain'] > pip_diff), 'direction'] = 'increase'
df.loc[(df['target_loss'] > df['target_gain']) & (df['target_loss'] > pip_diff), 'direction'] = 'decrease'

df['diff'] = df['close'].diff()

df['gain'] = 0
df['loss'] = 0
df['gain'].loc[df['diff'] > 0] = abs(df['diff'])
df['loss'].loc[df['diff'] < 0] = abs(df['diff'])

df['avg_gain'] = df['gain'].rolling(window=rsi_window).mean()
df['avg_loss'] = df['loss'].rolling(window=rsi_window).mean()

df['rs'] = df['avg_gain']/df['avg_loss']
df['rs'] = df['rs'].where(df['rs'] <= rs_max, rs_max) 

df['rsi'] = 100 - (100 / (df['rs'] + 1))

df['ssma'] = df['close'].rolling(window=window_s_ma).mean()
df['lsma'] = df['close'].rolling(window=window_l_ma).mean()
df['sma_diff'] = df['ssma'] - df['lsma']

df['sema'] = df['close'].ewm(span=window_s_ma).mean()
df['lema'] = df['close'].ewm(span=window_l_ma).mean()

df['ema_diff'] = df['sema'] - df['lema']

#df['slope_s'] = df['close'].rolling(window=window_s_ma).apply(get_slope_s)
#df['slope_l'] = df['close'].rolling(window=window_l_ma).apply(get_slope_l)
#df['sma_slope'] = df['ssma'].rolling(window=rsi_window).apply(get_slope)

df = df.dropna()
df = df.reset_index(drop=True)
df.head()

,open,high,low,close,target_gain,target_loss,direction,diff,gain,loss,avg_gain,avg_loss,rs,rsi,ssma,lsma,sma_diff,sema,lema,ema_diff
0,1.146625,1.146645,1.146595,1.146605,0.000035,0.000020,same,-0.000025,0.0,0.000025,0.000017,0.000014,1.214286,54.838710,1.146609,1.146387,0.000222,1.146591,1.146438,0.000152
1,1.146610,1.146640,1.146585,1.146605,0.000000,0.000030,same,0.000000,0.0,0.000000,0.000015,0.000014,1.107143,52.542373,1.146610,1.146385,0.000225,1.146593,1.146446,0.000148
2,1.146595,1.146605,1.146575,1.146600,0.000005,0.000030,same,-0.000005,0.0,0.000005,0.000015,0.000011,1.476190,59.615385,1.146615,1.146383,0.000232,1.146594,1.146453,0.000142
3,1.146590,1.146605,1.146570,1.146595,0.000010,0.000020,same,-0.000005,0.0,0.000005,0.000012,0.000011,1.090909,52.173913,1.146616,1.146386,0.000230,1.146595,1.146459,0.000136
4,1.146585,1.146605,1.146575,1.146590,0.000005,0.000055,same,-0.000005,0.0,0.000005,0.000008,0.000011,0.695652,41.025641,1.146613,1.146394,0.000219,1.146594,1.146465,0.000129


## Write data to csv

In [ ]:
#del df['TS']
df.to_csv(target_file_path, index = False)
winsound.PlaySound('C:\\Windows\\Media\\tada.wav', winsound.SND_ASYNC)

## Print Report

In [ ]:
target_col = 'direction'

print(f'Record count : {len(df)}')
print('--------------------------')
print(df[target_col].value_counts())
print('--------------------------')
print(df[target_col].value_counts(normalize=True))

df.head(10)